In [1]:
# default_exp data.load_pd

In [2]:
# hide
import sys

sys.path.append("..")
import pandas as pd

/home/tako/dev/env37/lib/python3.7/site-packages/pandas/compat/__init__.py:117: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [3]:
# export
from fastseq.core import *
from fastseq.data.external import *
from fastcore.utils import *
from fastcore.imports import *
from fastai2.basics import *
from fastai2.data.transforms import *
from fastai2.tabular.core import *

In [4]:
# hide
from nbdev.showdoc import *

# Data Load Dataframe


> Using the fastai2 `Datasets` to make an time series dataset.

A multivaraite time-series dataloader. 

TODO reduce mem: https://forums.fast.ai/t/how-to-handle-dataframes-too-large-to-fit-in-memory/39208/19

In [5]:
# export
import numpy as np
import pandas as pd

In [6]:
# export
class TSMulti(MultiTuple):pass

## Transform catagorical data

In [7]:
# export
class TensorCatI(TensorBase):pass
class CatSeqI(TensorSeqs):pass
def unpack_list(o, r=None):
    r = ifnone(r,L())
    for a in o:
        if isinstance(a,list) or isinstance(a,L):
            r = unpack_list(a, r)
        else:
            r.append(a)
    return r
            
class CatTfm(Transform):
    def __init__(self, df, cat_cols:[]): # maybe change to proccs
        self.vocab,self.o2i = {},{}
        for i, col in enumerate(L(cat_cols)):
            r = unpack_list(list(df[col]))
            self.vocab[i], self.o2i[i] = uniqueify(r, sort=True, bidir=True)
            
    def encodes(self, o:TensorCat):
        r = []
        for i in self.o2i:
            r.append(self.o2i[i][o.o[i]])            
        return TensorCatI(r, label = o._meta['label'])
    
    def decodes(self, o:TensorCatI): 
        r = []
        for i_cat in self.vocab:
            r.append(self.vocab[i_cat][o[i_cat]])
        return TensorCat(r, label = o._meta.get('label',None))
    
    def encodes(self, o:CatSeq):
        r = []
        for i in self.o2i:
            r.append([])
            for a in o.o[i]:
                r[i].append(self.o2i[i][a])            
        return CatSeqI(r, label = o._meta['label'])
    
    def decodes(self, o:CatSeqI):
        r = []
        for i in self.o2i:
            r.append([])
            for a in o[i]:
                r[i].append(self.vocab[i][a])  
        return CatSeq(r, label = o._meta.get('label',None))
    
    

In [8]:
df = pd.DataFrame({'a':[i for i in 'aabb']})
tmf = CatTfm(df, ['a'])
o_tmf = tmf(TensorCat(['a']))
test_eq_type(o_tmf,TensorCatI([0]))
test_eq(tmf.decode(o_tmf),TensorCat(['a']))

In [9]:
df = pd.DataFrame({'a':[i for i in 'aabb'], 'a_2':[i for i in 'bccb'],})
tmf = CatTfm(df, ['a','a_2'])
o_tmf = tmf(TensorCat(['a','b']))
test_eq_type(o_tmf,TensorCatI([0,0]))
test_eq(tmf.decode(o_tmf),TensorCat(['a','b']))

In [10]:
df = pd.DataFrame({'a':[[i,'a'] for i in 'aabb']})
tmf = CatTfm(df, ['a'])
o = CatSeq([['a','b','a','a']], label='a')
o_tmf = tmf(o)
print(o_tmf)
test_eq(o_tmf, TensorCatI([[0, 1, 0, 0]]))
test_eq(o_tmf._meta['label'], 'a')
dec_o = tmf.decode(o_tmf)
test_eq(dec_o,o)
dec_o

CatSeqI([[0, 1, 0, 0]])


CatSeq([['a', 'b', 'a', 'a']], label = ['a'])

In [11]:
df = pd.DataFrame({'a':[i for i in 'aabb'], 'a_2':[i for i in 'bccb'],})
tmf = CatTfm(df, ['a','a_2'])
o = CatSeq([['a','b','a','a'],['b','b','c','c']], label=['a','a_2'])
o_tmf = tmf(o)
print(o_tmf)
test_eq(o_tmf, CatSeqI([[0, 1, 0, 0],
                        [0, 0, 1, 1]]))
test_eq(o_tmf._meta['label'], ['a','a_2'])
dec_o = tmf.decode(o_tmf)
test_eq(dec_o,o)
dec_o

CatSeqI([[0, 1, 0, 0],
        [0, 0, 1, 1]])


CatSeq([['a', 'b', 'a', 'a'], ['b', 'b', 'c', 'c']], label = ['a', 'a_2'])

In [12]:
# export
class TSMulti_(Tuple):pass

class CatMultiTfm(ItemTransform):
    @delegates(CatTfm.__init__)
    def __init__(self, *args, **kwargs): # maybe change to proccs
        self.f = CatTfm(*args, **kwargs)
        
    def encodes(self, o:TSMulti):
        return TSMulti_(self.f(a) for a in o)
    
    def decodes(self, o:TSMulti_): 
        return TSMulti(self.f.decode(a) for a in o)
    

In [13]:
df = pd.DataFrame({'a':[i for i in 'aabb']})
tmf = CatMultiTfm(df,'a')
print(tmf)
o = TSMulti(TensorBase([1]), TensorCat('a'))
o_tmf = tmf(o)
test_eq(o_tmf,(TensorBase([1]), TensorCatI([0])) )
o_b =tmf.decode(o_tmf)
test_eq(o_b, o)
test_eq(type(o_b), type(TSMulti(1)))

CatMultiTfm: True (TSMulti,object) -> encodes (TSMulti_,object) -> decodes


# Dataloader

## pred

In [23]:
def get_df():
    return pd.DataFrame(data={'x':[np.arange(100.),
                           np.arange(120.)],
                   'con_ts_0':[np.ones(100)[None,:],
                           np.ones(120)[None,:]],
                   'con_ts_1':[pd.Series(np.arange(100)+np.random.randn(100)),
                           pd.Series(np.arange(120)+np.random.randn(120))],
                   'var_0':[0.,1.],
                   'var_1':[2.,1.5],
                   'cat_ts_0':[L(['a','b']*50),L(['b','a','b']*40)],
                   'cat_ts_1':[L(['y','z']*50),L(['x','y','z']*40)],
                   'con_0':['a','b'],
                   'con_1':['d','c']
                       })
df = get_df()
df

,x,con_ts_0,con_ts_1,var_0,var_1,cat_ts_0,cat_ts_1,con_0,con_1
0,"[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0, 20.0, 21.0, 22.0, 23.0, 24.0, 25.0, 26.0, 27.0, 28.0, 29.0, 30.0, 31.0, 32.0, 33.0, 34.0, 35.0, 36.0, 37.0, 38.0, 39.0, 40.0, 41.0, 42.0, 43.0, 44.0, 45.0, 46.0, 47.0, 48.0, 49.0, 50.0, 51.0, 52.0, 53.0, 54.0, 55.0, 56.0, 57.0, 58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65.0, 66.0, 67.0, 68.0, 69.0, 70.0, 71.0, 72.0, 73.0, 74.0, 75.0, 76.0, 77.0, 78.0, 79.0, 80.0, 81.0, 82.0, 83.0, 84.0, 85.0, 86.0, 87.0, 88.0, 89.0, 90.0, 91.0, 92.0, 93.0, 94.0, 95.0, 96.0, 97.0, 98.0, 99.0]","[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]]","0 -0.617810 1 1.428506 2 2.662515 3 2.044217 4 4.670755 ... 95 94.512024 96 93.138901 97 96.355890 98 98.152642 99 98.355421 Length: 100, dtype: float64",0.0,2.0,"[a, b, a, b, a, b, a, b, a, b, a, b, a, b, a, b, a, b, a, b, a, b, a, b, a, b, a, b, a, b, a, b, a, b, a, b, a, b, a, b, a, b, a, b, a, b, a, b, a, b, a, b, a, b, a, b, a, b, a, b, a, b, a, b, a, b, a, b, a, b, a, b, a, b, a, b, a, b, a, b, a, b, a, b, a, b, a, b, a, b, a, b, a, b, a, b, a, b, a, b]","[y, z, y, z, y, z, y, z, y, z, y, z, y, z, y, z, y, z, y, z, y, z, y, z, y, z, y, z, y, z, y, z, y, z, y, z, y, z, y, z, y, z, y, z, y, z, y, z, y, z, y, z, y, z, y, z, y, z, y, z, y, z, y, z, y, z, y, z, y, z, y, z, y, z, y, z, y, z, y, z, y, z, y, z, y, z, y, z, y, z, y, z, y, z, y, z, y, z, y, z]",a,d
1,"[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0, 20.0, 21.0, 22.0, 23.0, 24.0, 25.0, 26.0, 27.0, 28.0, 29.0, 30.0, 31.0, 32.0, 33.0, 34.0, 35.0, 36.0, 37.0, 38.0, 39.0, 40.0, 41.0, 42.0, 43.0, 44.0, 45.0, 46.0, 47.0, 48.0, 49.0, 50.0, 51.0, 52.0, 53.0, 54.0, 55.0, 56.0, 57.0, 58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65.0, 66.0, 67.0, 68.0, 69.0, 70.0, 71.0, 72.0, 73.0, 74.0, 75.0, 76.0, 77.0, 78.0, 79.0, 80.0, 81.0, 82.0, 83.0, 84.0, 85.0, 86.0, 87.0, 88.0, 89.0, 90.0, 91.0, 92.0, 93.0, 94.0, 95.0, 96.0, 97.0, 98.0, 99.0, ...]","[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...]]","0 -1.480290 1 1.635890 2 2.616556 3 4.088941 4 5.006840 ... 115 115.920240 116 116.319758 117 118.124653 118 117.602981 119 117.641192 Length: 120, dtype: float64",1.0,1.5,"[b, a, b, b, a, b, b, a, b, b, a, b, b, a, b, b, a, b, b, a, b, b, a, b, b, a, b, b, a, b, b, a, b, b, a, b, b, a, b, b, a, b, b, a, b, b, a, b, b, a, b, b, a, b, b, a, b, b, a, b, b, a, b, b, a, b, b, a, b, b, a, b, b, a, b, b, a, b, b, a, b, b, a, b, b, a, b, b, a, b, b, a, b, b, a, b, b, a, b, b, ...]","[x, y, z, x, y, z, x, y, z, x, y, z, x, y, z, x, y, z, x, y, z, x, y, z, x, y, z, x, y, z, x, y, z, x, y, z, x, y, z, x, y, z, x, y, z, x, y, z, x, y, z, x, y, z, x, y, z, x, y, z, x, y, z, x, y, z, x, y, z, x, y, z, x, y, z, x, y, z, x, y, z, x, y, z, x, y, z, x, y, z, x, y, z, x, y, z, x, y, z, x, ...]",b,c


In [24]:
# export
def array2series(o):
    return pd.Series(o.flatten())
def arrays2series(s:pd.Series):
    return pd.Series([array2series(o) for o in s])

In [25]:
df = pd.DataFrame(data={'x':  [np.arange(10.),
                               np.arange(12.)],
                       'ts_0':[np.ones(10)[None,:],
                               np.ones(12)[None,:]],
                       })
df['x'] = arrays2series(df['x'])
df['ts_0'] = arrays2series(df['ts_0'])
test_eq(type(df['ts_0']), pd.Series)
test_eq(type(df['x']), pd.Series)

In [26]:
# hide
df = pd.DataFrame(data={'x':  [np.arange(10.)]*100,
                       'ts_0':[np.ones(10)[None,:]]*100,
                       })
df['x'] = arrays2series(df['x'])
df['ts_0'] = arrays2series(df['ts_0'])
test_eq(type(df['ts_0']), pd.Series)
test_eq(type(df['x']), pd.Series)

In [27]:
# export
def reconize_cols(dataset):
    con_names, cat_names, con_ts_names, cat_ts_names, classes = L(), L(), L(), L(), {}
    for col in dataset.columns:
        t = type(dataset[col].iloc[0])
        if t is pd.core.series.Series:
            con_ts_names.append(col)
        elif t is np.ndarray:
            o = arrays2series(dataset[col])
            dataset[col] = o
            con_ts_names.append(col)
        elif isinstance(dataset[col].iloc[0], str):
            cat_names.append(col)
            classes[col] = uniqueify(list(dataset[col].values))
        elif isinstance(dataset[col].iloc[0], float) or isinstance(dataset[col].iloc[0], int) or t is np.int64:
            con_names.append(col)            
        elif isinstance(dataset[col].iloc[0], L) or isinstance(dataset[col].iloc[0],list):
            cat_ts_names.append(col)
            classes[col] = uniqueify(unpack_list(list(dataset[col])))
        else:
            raise Exception(t)
    return con_names, cat_names, con_ts_names, cat_ts_names, classes, dataset

In [35]:
df = get_df()
con_names, cat_names, con_ts_names, cat_ts_names, classes, dataset = reconize_cols(df)
test_eq(con_names, ['var_0','var_1'])
test_eq(cat_names, ['con_0','con_1'])
test_eq(con_ts_names, ['x','con_ts_0','con_ts_1'])
test_eq(cat_ts_names, ['cat_ts_0','cat_ts_1'])
test_eq(classes, {'cat_ts_0': ['a','b'],
                  'cat_ts_1': ['y','z','x'],
                  'con_0': ['a','b'],
                  'con_1': ['d','c']
                 })
con_names, cat_names, con_ts_names, cat_ts_names

((#2) ['var_0','var_1'],
 (#2) ['con_0','con_1'],
 (#3) ['x','con_ts_0','con_ts_1'],
 (#2) ['cat_ts_0','cat_ts_1'])

In [50]:
list([L([a for a in o]) for o in df[['cat_ts_0','cat_ts_1']].values])

[(#2) [(#100) ['a','b','a','b','a','b','a','b','a','b'...],(#100) ['y','z','y','z','y','z','y','z','y','z'...]],
 (#2) [(#120) ['b','a','b','b','a','b','b','a','b','b'...],(#120) ['x','y','z','x','y','z','x','y','z','x'...]]]

In [56]:
# export
class PrepDF(TabularProc):
    def setup(self, dl, train_setup):
        # speed up retrival
        dl.con = dl.dataset.loc[:,dl.con_names].values.astype(float)
        dl.cat = [list(dl.dataset.loc[i,dl.cat_names]) for i in range(dl.dataset.shape[0])]
        if len(dl.con_ts_names):
            dl.tsx_con = [np.concatenate([o[None,:] for o in dl.dataset.loc[i,dl.con_ts_names].values]) for i in range(dl.dataset.shape[0])]
        else:
            dl.tsx_con = [np.empty([0,0]) for i in range(dl.dataset.shape[0])]
            
        if len(dl.cat_ts_names):
            dl.tsx_cat = list([L([a for a in o]) for o in df[dl.cat_ts_names].values])
        else:
            dl.tsx_cat = [[]]            
        assert len(dl.cat) == len(dl.tsx_con) == dl.con.shape[0] == len(dl.tsx_cat)
        return dl

In [66]:
class test_dl(TfmdDL):    
    def __init__(self, dataset:pd.DataFrame, y_name ):
        con_names, cat_names, con_ts_names, cat_ts_names, classes, dataset = reconize_cols(dataset)  
        store_attr(self,'y_name,con_names,cat_names,con_ts_names,cat_ts_names,classes,dataset')     
        assert y_name in self.con_ts_names, {k:getattr(self,k) for k in 'con_names,cat_names,con_ts_names,cat_ts_names'.split(',')}
        self.con_ts_names.remove(y_name) 
        PrepDF().setup(self, True)        
df = get_df()
dl = test_dl(df, 'x')
test_eq(dl.cat,[['a', 'd'], ['b', 'c']])
test_eq(dl.con.shape,(2,2))
test_eq(dl.tsx_con[0].shape, [2,100])
test_eq(len(dl.tsx_cat), 2)
test_eq(len(dl.tsx_cat[0]), 2)
test_eq(len(dl.tsx_cat[0][0]), 100)
dl.tsx_cat

[(#2) [(#100) ['a','b','a','b','a','b','a','b','a','b'...],(#100) ['y','z','y','z','y','z','y','z','y','z'...]],
 (#2) [(#120) ['b','a','b','b','a','b','b','a','b','b'...],(#120) ['x','y','z','x','y','z','x','y','z','x'...]]]

In [67]:
# hide
from nbdev.export import *
notebook2script()

Converted 00_core.ipynb.
Converted 01_data.external.ipynb.
Converted 020_data.load_pd.ipynb.
Converted 02_data.load.ipynb.
Converted 030_data.core_pd.ipynb.
Converted 03_data.core.ipynb.
Converted 050_nbeats.models_multi.ipynb.
Converted 05_nbeats.models.ipynb.
Converted 06_nbeats.callbacks.ipynb.
Converted 07_nbeats.learner.ipynb.
Converted 08_nbeats.interpret.ipynb.
Converted 11_metrics.ipynb.
Converted 12_compare.ipynb.
Converted index.ipynb.
Converted tab.model.ipynb.
